# <strong>5: ELAN Annotation Notebook |</strong>


### <strong>Requirements</strong>

To run this notebook, you will need the following Python packages:

- pandas

In [ ]:
import os
import pandas as pd

In [ ]:
input_directory = '../Gestures/'  # Replace with your directory path
output_directory = '../Annotations/'  # Replace with your desired output directory
tier = 'gesture_id'  # Replace with your desired tier name

### <strong>Export Apex Annotations</strong>

In [ ]:
# Iterate through all CSV files in the directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):
        # Read the CSV file
        file_path = os.path.join(input_directory, filename)
        gest_df = pd.read_csv(file_path)

        gest_df['change_points'] = gest_df[tier].shift(1) != gest_df[tier]
        gest_df['time_ms'] = gest_df['time_ms'] / 1000
        grouped_gest_df = gest_df.groupby((gest_df['change_points']).cumsum())
        gest_df = pd.DataFrame({
            'Begin Time': grouped_gest_df['time_ms'].first(),
            'End Time': grouped_gest_df['time_ms'].last(),
            tier: grouped_gest_df[tier].first(),
        })
        gest_df = gest_df.dropna(subset=[tier])
        gest_df = gest_df[gest_df[tier] != 0]
        gest_df['End Time'] = gest_df['End Time'] + 0.033

        # Save the modified DataFrame
        output_file_path = os.path.join(output_directory, filename.split('.')[0] + f'_elan_{tier}.csv')
        gest_df.to_csv(output_file_path, index=False)

In [ ]:
gest_df.head(20)